In [1]:
# pip install -U funasr
'''
Using the finetuned emotion recognization model

rec_result contains {'feats', 'labels', 'scores'}
	extract_embedding=False: 9-class emotions with scores
	extract_embedding=True: 9-class emotions with scores, along with features

9-class emotions:
iic/emotion2vec_plus_seed, iic/emotion2vec_plus_base, iic/emotion2vec_plus_large (May. 2024 release)
iic/emotion2vec_base_finetuned (Jan. 2024 release)
    0: angry
    1: disgusted
    2: fearful
    3: happy
    4: neutral
    5: other
    6: sad
    7: surprised
    8: unknown
'''

from funasr import AutoModel

# model="iic/emotion2vec_base"
# model="iic/emotion2vec_base_finetuned"
# model="iic/emotion2vec_plus_seed"
# model="iic/emotion2vec_plus_base"
model_id = "iic/emotion2vec_plus_large"

model = AutoModel(
    model=model_id,
    hub="ms",  # "ms" or "modelscope" for China mainland users; "hf" or "huggingface" for other overseas users
)




2025-03-13 21:09:42,271 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


Detect model requirements, begin to install it: C:\Users\19389\.cache\modelscope\hub\models\iic\emotion2vec_plus_large\requirements.txt
install model requirements successfully
ckpt: C:\Users\19389\.cache\modelscope\hub\models\iic\emotion2vec_plus_large\model.pt
init param, map: modality_encoders.AUDIO.extra_tokens from d2v_model.modality_encoders.AUDIO.extra_tokens in ckpt
init param, map: modality_encoders.AUDIO.alibi_scale from d2v_model.modality_encoders.AUDIO.alibi_scale in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias in ckpt
init param, map: modal

In [5]:
# wav_file = f"{model.model_path}/example/test.wav"
wav_file = "../data/voice/1DCnIad1Y0w.002.wav"


In [6]:
rec_result = model.generate(wav_file, output_dir="./outputs", granularity="utterance", extract_embedding=False)
print(rec_result)

rtf_avg: 0.148: 100%|██████████| 1/1 [00:02<00:00,  2.26s/it]                                                                                      

[{'key': 'output', 'labels': ['生气/angry', '厌恶/disgusted', '恐惧/fearful', '开心/happy', '中立/neutral', '其他/other', '难过/sad', '吃惊/surprised', '<unk>'], 'scores': [3.12398998542681e-14, 2.9636185270177223e-14, 4.654729377683237e-15, 1.0, 2.887142902707315e-17, 3.583470821543844e-17, 3.3797961377832718e-15, 3.8882986457336455e-14, 1.3343034939560418e-24]}]


In [7]:
#找到字典中scorse最高对应的labels
max_score = max(rec_result[0]['scores'])
max_index = rec_result[0]['scores'].index(max_score)
max_label = rec_result[0]['labels'][max_index]
print(f"The most possible emotion is {max_label} with score {max_score}")

The most possible emotion is 开心/happy with score 1.0


In [29]:
from playsound import playsound
playsound(wav_file)

ERROR:playsound:
    Error 263 for command:
        open ../data/wav/test.wav
    指定的设备未打开，或不被 MCI 所识别。
ERROR:playsound:
    Error 263 for command:
        close ../data/wav/test.wav
    指定的设备未打开，或不被 MCI 所识别。


PlaysoundException: 
    Error 263 for command:
        open ../data/wav/test.wav
    指定的设备未打开，或不被 MCI 所识别。

# 看看两个音频文件有什么不同

In [5]:
import librosa

def get_audio_properties(file_path):
    # 加载音频文件
    y, sr = librosa.load(file_path, sr=None)
    # 获取音频时长
    duration = librosa.get_duration(y=y, sr=sr)
    # 获取通道数，librosa默认读取为单声道，若要获取原始通道数，需要额外处理
    channels = 1 if len(y.shape) == 1 else y.shape[1]
    return {'sample_rate': sr, 'channels': channels, 'duration': duration}

def compare_audio_files(file1, file2):
    props1 = get_audio_properties(file1)
    props2 = get_audio_properties(file2)

    print("File 1 properties:", props1)
    print("File 2 properties:", props2)

    # 比较属性
    for key in props1.keys():
        if props1[key] != props2[key]:
            print(f"Difference found in {key}: File 1 has {props1[key]}, File 2 has {props2[key]}")
        else:
            print(f"{key.capitalize()} is the same.")

# 替换成你的音频文件路径
file1 = "../data/wav/test.wav"
file2 = f"{model.model_path}/example/test.wav"
compare_audio_files(file1, file2)

File 1 properties: {'sample_rate': 48000, 'channels': 1, 'duration': 3.9372708333333333}
File 2 properties: {'sample_rate': 16000, 'channels': 1, 'duration': 4.104125}
Difference found in sample_rate: File 1 has 48000, File 2 has 16000
Channels is the same.
Difference found in duration: File 1 has 3.9372708333333333, File 2 has 4.104125
